# Build the logbook for Hologram with empty or filters

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : December 8th 2022
- last update : July 19th 2023


Works directory per directory (day by day)

CCIN2P3:


- works with the new kernel **miniconda3_py310_auxtel** (with stackyter.py) 
- works with the new kernel **miniconda3_py38_auxtel** (with notebook.cc.in2p3.fr) 



REMEMBER:
- to_excel() uses a library called **xlwt** and **openpyxl** internally. openpyxl is used to write .

In [1]:
import sys
sys.path

['/pbs/throng/lsst/users/dagoret/desc/AuxTelComm/notebookccdm_gen3/runspectractor_standalone/2023_07',
 '',
 '/opt/conda/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/miniconda3-py38/lib/python38.zip',
 '/sps/lsst/groups/auxtel/softs/utils/miniconda3-py38/lib/python3.8',
 '/sps/lsst/groups/auxtel/softs/utils/miniconda3-py38/lib/python3.8/lib-dynload',
 '/pbs/home/d/dagoret/.local/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/miniconda3-py38/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/miniconda3-py38/lib/python3.8/site-packages/Spectractor-2.5-py3.8.egg']

In [2]:
! ls /sps/lsst/groups/auxtel/data/hack_usdf/my_postisrccd_img_forspectractor_2023/empty~holo4_003
#! ls /Users/sylvie/DATA/AuxTelDATA2023/data/2023/empty~holo4_003

20221212  20230118  20230131  20230202	20230315  20230704  20230706
20230117  20230119  20230201  20230215	20230316  20230705


# Select the combination DATE, FILTER, DISPERSER

In [3]:
DATE = '20230119'
filterdispersernames = ["empty~holo4_003","BG40~holo4_003","FELH0600~holo4_003","SDSSg~holo4_003",\
                        "empty~ronchi170lpmm","BG40~ronchi170lpmm","FELH0600~ronchi170lpmm","SDSSg~ronchi170lpmm",\
                       ]
filterdispersername = filterdispersernames[0]
version="v6.0"  # new orientation

# If position need to be specified add it in this dictionnary

In [4]:
# reconstructed / good /xpos /ypos

if version == "v6.0":
    info={\

          # with red filter
          'exposure_2023031600351_postisrccd.fits':(1,1,300.,1700),\
         }
        

# Imports

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [6]:
from astropy.io import fits

In [7]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

/tmp/ipykernel_3323/3547059130.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [8]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [9]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [10]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [11]:
dir_logbooks="logbooks"

In [12]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [13]:
home=os.getenv("HOME")

In [14]:
image_path=filterdispersername
logbook_csv=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_"+ DATE +"_" + version + ".csv")
logbook_xlsx=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_" +DATE +"_" + version + ".xlsx")

# Path

In [15]:
HOSTCC=True
LAPTOP=False

In [16]:
# Set path depending on which computer running (according HOSTCC)
#/sps/lsst/groups/auxtel/data/hack_usdf/my_postisrccd_img_forspectractor_2023/empty~holo4_003
if HOSTCC:
    path_auxtel="/sps/lsst/groups/auxtel"
    path_spectractor=os.path.join(path_auxtel,"softs/github/desc/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    #path_images=os.path.join(path_auxtel,"data/2023/"+filterdispersername+"/"+DATE)
    path_images=os.path.join(path_auxtel,"data/hack_usdf/my_postisrccd_img_forspectractor_2023/"+filterdispersername+"/"+DATE)

else:
    if not LAPTOP:
        #path_auxtel="/Users/sylvie/DATA/AuxTelData2022"
        path_auxtel="/Users/sylvie/DATA/AuxTelData2023"
        path_spectractor=os.path.join(path_auxtel,"/Users/sylvie/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        #path_images=os.path.join(path_auxtel,"data/2022/"+filterdispersername+"/"+DATE)
        path_images=os.path.join(path_auxtel,"data/2023/"+filterdispersername+"/"+DATE)
        
    else:
        #path_auxtel="/Users/dagoret/DATA/AuxTelData2022"
        path_auxtel="/Users/dagoret/DATA/AuxTelData2023"
        path_spectractor=os.path.join(path_auxtel,"/Users/dagoret/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2023/"+filterdispersername+"/"+DATE)
        

# Check the images exists

In [17]:
dir_images=path_images

In [18]:
all_files=os.listdir(dir_images)

In [19]:
all_files

['exposure_2023011900344_pseudo-postisrccd.fits',
 'exposure_2023011900163_pseudo-postisrccd.fits',
 'exposure_2023011900384_pseudo-postisrccd.fits',
 'exposure_2023011900314_pseudo-postisrccd.fits',
 'exposure_2023011900227_pseudo-postisrccd.fits',
 'exposure_2023011900362_pseudo-postisrccd.fits',
 'exposure_2023011900327_pseudo-postisrccd.fits',
 'exposure_2023011900278_pseudo-postisrccd.fits',
 'exposure_2023011900288_pseudo-postisrccd.fits',
 'exposure_2023011900315_pseudo-postisrccd.fits',
 'exposure_2023011900306_pseudo-postisrccd.fits',
 'exposure_2023011900226_pseudo-postisrccd.fits',
 'exposure_2023011900363_pseudo-postisrccd.fits',
 'exposure_2023011900180_pseudo-postisrccd.fits',
 'exposure_2023011900296_pseudo-postisrccd.fits',
 'exposure_2023011900375_pseudo-postisrccd.fits',
 'exposure_2023011900190_pseudo-postisrccd.fits',
 'exposure_2023011900335_pseudo-postisrccd.fits',
 'exposure_2023011900279_pseudo-postisrccd.fits',
 'exposure_2023011900383_pseudo-postisrccd.fits',


In [20]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [21]:
all_selected_files=np.array(all_selected_files)

### Sort files

In [22]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('exposure.*_(.*)_pseudo-postisrccd[.]fits$',filename)
    
    return m[0]

In [23]:
file_tag_forsorting('exposure_2022031700310_pseudo-postisrccd.fits')

'2022031700310'

In [24]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['2023011900344', '2023011900163', '2023011900384', '2023011900314',
       '2023011900227', '2023011900362', '2023011900327', '2023011900278',
       '2023011900288', '2023011900315', '2023011900306', '2023011900226',
       '2023011900363', '2023011900180', '2023011900296', '2023011900375',
       '2023011900190', '2023011900335', '2023011900279', '2023011900383',
       '2023011900392', '2023011900218', '2023011900354', '2023011900402',
       '2023011900199', '2023011900401', '2023011900336', '2023011900353',
       '2023011900297', '2023011900305', '2023011900217', '2023011900269',
       '2023011900374', '2023011900254', '2023011900270', '2023011900393',
       '2023011900287', '2023011900326', '2023011900189', '2023011900345',
       '2023011900198', '2023011900181', '2023011900255'], dtype='<U13')

In [25]:
indexes=np.argsort(all_tags)

In [26]:
sorted_files=all_selected_files[indexes]

In [27]:
sorted_files

array(['exposure_2023011900163_pseudo-postisrccd.fits',
       'exposure_2023011900180_pseudo-postisrccd.fits',
       'exposure_2023011900181_pseudo-postisrccd.fits',
       'exposure_2023011900189_pseudo-postisrccd.fits',
       'exposure_2023011900190_pseudo-postisrccd.fits',
       'exposure_2023011900198_pseudo-postisrccd.fits',
       'exposure_2023011900199_pseudo-postisrccd.fits',
       'exposure_2023011900217_pseudo-postisrccd.fits',
       'exposure_2023011900218_pseudo-postisrccd.fits',
       'exposure_2023011900226_pseudo-postisrccd.fits',
       'exposure_2023011900227_pseudo-postisrccd.fits',
       'exposure_2023011900254_pseudo-postisrccd.fits',
       'exposure_2023011900255_pseudo-postisrccd.fits',
       'exposure_2023011900269_pseudo-postisrccd.fits',
       'exposure_2023011900270_pseudo-postisrccd.fits',
       'exposure_2023011900278_pseudo-postisrccd.fits',
       'exposure_2023011900279_pseudo-postisrccd.fits',
       'exposure_2023011900287_pseudo-postisrccd

In [28]:
N=len(sorted_files)

# Build the logbook from the headers

In [29]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=hdr0['OBJECT']
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [30]:
df=pd.DataFrame()

In [31]:
len(all_airmass)

43

In [32]:
N=len(all_indexes)

## Fill with container info

In [33]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [34]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [35]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [36]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [37]:
pd.set_option('display.max_rows', None)

In [38]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2023-01-20T01:01:40.537,/sps/lsst/groups/auxtel/data/hack_usdf/my_post...,exposure_2023011900163_pseudo-postisrccd.fits,163,HD185975,empty_1,holo4_003,2.043105,30.0,...,10.0,50.0,5.0,NaN,NaN,NaN,NaN,,NaN,NaN
1,1,2023-01-20T01:16:29.941,/sps/lsst/groups/auxtel/data/hack_usdf/my_post...,exposure_2023011900180_pseudo-postisrccd.fits,180,HD37962,empty_1,holo4_003,1.037663,30.0,...,10.0,50.0,5.0,NaN,NaN,NaN,NaN,,NaN,NaN
2,2,2023-01-20T01:17:12.778,/sps/lsst/groups/auxtel/data/hack_usdf/my_post...,exposure_2023011900181_pseudo-postisrccd.fits,181,HD37962,empty_1,holo4_003,1.036902,30.0,...,10.0,50.0,5.0,NaN,NaN,NaN,NaN,,NaN,NaN
3,3,2023-01-20T01:21:52.976,/sps/lsst/groups/auxtel/data/hack_usdf/my_post...,exposure_2023011900189_pseudo-postisrccd.fits,189,HD37962,empty_1,holo4_003,1.032129,30.0,...,10.0,50.0,5.0,NaN,NaN,NaN,NaN,,NaN,NaN
4,4,2023-01-20T01:22:35.897,/sps/lsst/groups/auxtel/data/hack_usdf/my_post...,exposure_2023011900190_pseudo-postisrccd.fits,190,HD37962,empty_1,holo4_003,1.031428,30.0,...,10.0,50.0,5.0,NaN,NaN,NaN,NaN,,NaN,NaN
5,5,2023-01-20T01:28:47.949,/sps/lsst/groups/auxtel/data/hack_usdf/my_post...,exposure_2023011900198_pseudo-postisrccd.fits,198,HD031128,empty_1,holo4_003,1.002421,30.0,...,10.0,50.0,5.0,NaN,NaN,NaN,NaN,,NaN,NaN
6,6,2023-01-20T01:29:30.757,/sps/lsst/groups/auxtel/data/hack_usdf/my_post...,exposure_2023011900199_pseudo-postisrccd.fits,199,HD031128,empty_1,holo4_003,1.002306,30.0,...,10.0,50.0,5.0,NaN,NaN,NaN,NaN,,NaN,NaN
7,7,2023-01-20T01:45:34.527,/sps/lsst/groups/auxtel/data/hack_usdf/my_post...,exposure_2023011900217_pseudo-postisrccd.fits,217,HD185975,empty_1,holo4_003,2.070875,30.0,...,10.0,50.0,5.0,NaN,NaN,NaN,NaN,,NaN,NaN
8,8,2023-01-20T01:46:17.334,/sps/lsst/groups/auxtel/data/hack_usdf/my_post...,exposure_2023011900218_pseudo-postisrccd.fits,218,HD185975,empty_1,holo4_003,2.071294,30.0,...,10.0,50.0,5.0,NaN,NaN,NaN,NaN,,NaN,NaN
9,9,2023-01-20T01:51:23.804,/sps/lsst/groups/auxtel/data/hack_usdf/my_post...,exposure_2023011900226_pseudo-postisrccd.fits,226,HD14943,empty_1,holo4_003,1.238150,30.0,...,10.0,50.0,5.0,NaN,NaN,NaN,NaN,,NaN,NaN


# Make logbook


In [39]:
N=len(df)
N

43

In [40]:
if N>0:
    df.to_excel(logbook_xlsx)
    df.to_csv(logbook_csv)

In [41]:
logbook_csv

'logbooks/auxtellogbook_empty~holo4_003_20230119_v6.0.csv'